In [14]:
import torch
import pennylane as qml
from time import time
from tqdm import tqdm
import numpy as np
import matplotlib as plt
from circuit_model_II import QuantumCircuit, FullQuantumModel
import torch
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import numpy as np
import os
from torch.utils.data import DataLoader, random_split

In [2]:
num_qubits = 8
num_layers = 5
model = FullQuantumModel(num_qubits, num_layers)

In [ ]:
model.draw(style)

# Dataset preparation

In [12]:
# Download MNIST and prepare transforms
mnist_train = datasets.MNIST(root='./data', train=True, download=True,
                             transform=transforms.Compose([
                                transforms.Resize((16, 16)),  # Resize to 16x16
                                transforms.ToTensor(),
                                transforms.Normalize((0.1307,), (0.3081,))  # Normalize
                             ]))
# Filter for zeros and ones
data = []
targets = []
for image, label in mnist_train:
    if label in [0, 1]:
        data.append(image.squeeze())
        targets.append(label)

data = torch.stack(data)
targets = torch.tensor(targets)
# Select 1024 zeros and 1024 ones for speed
zeros_indices = (targets == 0)
ones_indices = (targets == 1)

zeros = data[zeros_indices]
ones = data[ones_indices]

#normalize between 0 and 1
zeros_max = torch.max(zeros.reshape(-1, 16*16), dim = 1)
zeros_min = torch.min(zeros.reshape(-1, 16*16), dim = 1)
ones_max = torch.max(ones.reshape(-1, 16*16), dim = 1)
ones_min = torch.min(ones.reshape(-1, 16*16), dim = 1)

def normalize(imgs):
  maxes, _ = torch.max(imgs.reshape(-1, 16*16), dim = 1)
  mins, _ = torch.min(imgs.reshape(-1, 16*16), dim = 1)

  mins = mins.unsqueeze(1).unsqueeze(2)
  maxes = maxes.unsqueeze(1).unsqueeze(2)

  return (imgs-mins)/(maxes-mins)

zeros = normalize(zeros)
ones = normalize(ones)

# assert images have min 0 and max 1 within an error of 1e-5
assert torch.allclose(zeros.min(), torch.tensor(0., dtype = torch.float32), atol=1e-5)
assert torch.allclose(zeros.max(), torch.tensor(1., dtype = torch.float32), atol=1e-5)
assert torch.allclose(ones.min(), torch.tensor(0., dtype = torch.float32), atol=1e-5)
assert torch.allclose(ones.max(), torch.tensor(1., dtype = torch.float32), atol=1e-5)

# concatenate the two datasets
zeros = zeros.flatten(start_dim = 1)
ones = ones.flatten(start_dim = 1)
dataset = torch.cat((zeros, ones), dim = 0)

# add labels
labels = torch.cat((torch.zeros((zeros.shape[0], 1)), torch.ones((ones.shape[0], 1))), dim = 0).squeeze()

# build dataloader
dataset = torch.utils.data.TensorDataset(dataset, labels)

In [15]:
#train/test split

train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size

train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

BATCH_SIZE = 32

train_dataloader = DataLoader(train_dataset, batch_size = BATCH_SIZE, shuffle = True, drop_last = True)

test_dataloader = DataLoader(test_dataset, batch_size = BATCH_SIZE, shuffle = True, drop_last = True)

In [17]:
len(test_dataset)

2533

# Model training

In [8]:
type(model.fit(dataloader=train_dataloader, learning_rate=0.01, epochs=1, num_layers_to_execute = 2))

Epoch 1/1: 100%|██████████| 64/64 [00:01<00:00, 46.14it/s, accuracy=0.969, loss=0.445]

Time per epoch:  1.410733699798584
Epoch:  0 Loss:  0.4453395903110504
Accuracy:  0.96875
--------------------------------------------------------------------------


tuple

In [11]:
for i in model.parameters():
    print(i)

Parameter containing:
tensor([[ 6.4518e-01,  2.7357e-01,  1.3663e+00],
        [-3.5419e-01,  7.6826e-01,  1.1332e+00],
        [ 3.0931e-01, -3.1503e-01,  2.2945e-01],
        [ 9.5090e-01,  6.5840e-01,  1.3437e+00],
        [ 5.4103e-01, -1.6942e-02,  3.6131e-01],
        [ 1.1233e+00,  2.7824e-01,  4.4224e-01],
        [ 5.5193e-01,  1.0022e-03,  4.9343e-01],
        [ 1.3166e-01,  1.1371e+00,  5.5685e-01]], requires_grad=True)
Parameter containing:
tensor([[ 0.8050,  0.5232,  0.0579],
        [ 0.9479,  0.1219,  0.3449],
        [ 1.0585,  0.7306,  0.8181],
        [ 1.3893, -0.3028,  0.9060],
        [ 0.9597,  0.1089,  0.5592],
        [ 0.8824,  0.0786,  0.8563],
        [ 1.4634,  0.0185,  0.2640],
        [ 0.3201,  1.3131,  0.7878]], requires_grad=True)
Parameter containing:
tensor([[0.8994, 0.4576, 0.6736],
        [0.3294, 0.2796, 0.3973],
        [0.6277, 0.0261, 0.2352],
        [0.9228, 0.7249, 0.3469],
        [0.0523, 0.6643, 0.7118],
        [0.0241, 0.1838, 0.6101],
